In [28]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers


In [29]:
data = pd.read_csv("kc_house_data.csv")
data.shape

(21613, 21)

In [30]:
# Display data
pd.options.display.max_columns = 25
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [31]:
# Display data types
data.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [32]:
# Data preprocess (pdf.72)
data['year'] = pd.to_numeric(data['date'].str.slice(0, 4))
data['month'] = pd.to_numeric(data['date'].str.slice(4, 6))
data['day'] = pd.to_numeric(data['date'].str.slice(6, 8))

data.drop(['id'], axis="columns", inplace=True)
data.drop(['date'], axis="columns", inplace=True)
data.head()


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,2014,10,13
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2014,12,9
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,2015,2,25
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,2014,12,9
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,2015,2,18


In [33]:
# Split dataset to 3 parts (pdf.73)
data_num = data.shape[0]
# 打散資料 : permutation() 返回打亂後的array
indexes = np.random.permutation(data_num)

train_indexes = indexes[:int(data_num * 0.6)]
val_indexes = indexes[int(data_num * 0.6):int(data_num * 0.8)]
test_indexes = indexes[int(data_num * 0.8):]

train_data = data.loc[train_indexes]
val_data = data.loc[train_indexes]
test_data = data.loc[train_indexes]


In [34]:
# for test 
locs = [1, 3, 5]
data_locs = data.loc[locs]
print(data_locs)

       price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
1   538000.0         3       2.25         2570      7242     2.0           0   
3   604000.0         4       3.00         1960      5000     1.0           0   
5  1225000.0         4       4.50         5420    101930     1.0           0   

   view  condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
1     0          3      7        2170            400      1951          1991   
3     0          5      7        1050            910      1965             0   
5     0          3     11        3890           1530      2001             0   

   zipcode      lat     long  sqft_living15  sqft_lot15  year  month  day  
1    98125  47.7210 -122.319           1690        7639  2014     12    9  
3    98136  47.5208 -122.393           1360        5000  2014     12    9  
5    98053  47.6561 -122.005           4760      101930  2014      5   12  


In [35]:
# Normalization (pdf.73)
"""
Z-score :
    Xnorm = (X - mean) / std
"""
train_validation_data = pd.concat([train_data, val_data])
mean = train_validation_data.mean()
std = train_validation_data.std()
train_data = (train_data - mean) / std
val_data = (val_data - mean) /std


In [36]:
# Establish Numpy array type's training sets(
#print(train_data)

x_train = np.array(train_data.drop('price', axis='columns')) # drop "price" column
y_train = np.array(train_data['price'])
x_val = np.array(val_data.drop('price', axis='columns'))
y_val = np.array(val_data['price'])

#print(y_train)

In [37]:
x_train.shape

(12967, 21)

In [27]:
# Establish training network model (pdf.74)
model = keras.Sequential(name=' model-1')
model.add(layers.Dense(64, activation='relu', input_shape=(21,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

model.summary()

2023-10-29 14:31:59.088831: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-10-29 14:31:59.110150: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 14:31:59.138252: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: " model-1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1408      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,633
Trainable params: 5,633
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Set optimizers, losses fun, metrics fun (pdf.74)
model.compile(keras.optimizers.Adam(0.001),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.MeanAbsoluteError()])

In [ ]:
# Set model dir
# ?? 查目前所在目錄，再調整下面參數
model_dir = 'lab2-logs/models/'
os.makedirs(model_dir)


In [ ]:
# Set Callback fun (pdf.75)
log_dir = os.path.join('lab2-logs', 'model-1')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(model_dir + 'Best-model-1.h5',
                                             monitor='val_mean_absolute_error',
                                             save_best_only=True,
                                             mode='min)

In [ ]:
# Training network model (pdf.75)
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=300,
                    validation_data=(x_val, y_val),
                    callbacks=[model_cbk, model_mckp])


In [ ]:
# History record
history.history.keys()

In [ ]:
# Draw pic
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')


In [ ]:
# Draw metrics index
plt.plot(history.history['mean_absolute_error'], label='train')
plt.plot(history.history['val_mean_absolute_error'], label='validation')
plt.ylabel('metrics')
pltxlabel('epochs')
plt.legend(loc='upper right')